# Ensemble Model

#  Load Pre-trained Models

## 1. Text Model (mBERT)

In [ ]:
# =============================
# LOAD MBERT MODEL
# =============================
print("Loading mBERT model...")
mbert_tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
mbert_model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=2
)

# Load trained weights
mbert_model.load_state_dict(torch.load(MBERT_MODEL_PATH, map_location=device))
mbert_model = mbert_model.to(device)
mbert_model.eval()

print("✓ mBERT model loaded successfully!")

## 2. Visual Model (CLIP)

In [ ]:
# =============================
# LOAD CLIP MODEL
# =============================
print("Loading CLIP model...")

# CLIP Classifier class (same as training)
class CLIPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes=2, dropout=0.3):
        super(CLIPClassifier, self).__init__()
        self.clip = clip_model
        hidden_size = self.clip.config.projection_dim
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_classes)
        )
    
    def forward(self, pixel_values):
        vision_outputs = self.clip.vision_model(pixel_values=pixel_values)
        image_embeds = vision_outputs.pooler_output
        image_embeds = self.clip.visual_projection(image_embeds)
        logits = self.classifier(image_embeds)
        return logits

# Load CLIP
clip_base = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model = CLIPClassifier(clip_base, num_classes=2)

# Load trained weights
clip_model.load_state_dict(torch.load(CLIP_MODEL_PATH, map_location=device))
clip_model = clip_model.to(device)
clip_model.eval()

print("✓ CLIP model loaded successfully!")

#  Generate Predictions from Base Models

## Helper Functions

In [ ]:
# =============================
# PREDICTION FUNCTIONS
# =============================

def get_text_predictions(texts, model, tokenizer, device, batch_size=16):
    """Get probability predictions from text model (mBERT)"""
    model.eval()
    all_probs = []
    
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Text predictions"):
            batch_texts = texts[i:i+batch_size]
            
            # Tokenize
            encoding = tokenizer(
                batch_texts,
                add_special_tokens=True,
                max_length=MAX_LENGTH,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            
            input_ids = encoding['input_ids'].to(device)
            attention_mask = encoding['attention_mask'].to(device)
            
            # Get predictions
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            probs = torch.softmax(outputs.logits, dim=1)
            all_probs.append(probs.cpu().numpy())
    
    return np.vstack(all_probs)


def get_image_predictions(image_names, img_dir, model, processor, device, batch_size=16):
    """Get probability predictions from visual model (CLIP)"""
    model.eval()
    all_probs = []
    
    with torch.no_grad():
        for i in tqdm(range(0, len(image_names), batch_size), desc="Image predictions"):
            batch_names = image_names[i:i+batch_size]
            batch_images = []
            
            # Load images
            for img_name in batch_names:
                img_path = os.path.join(img_dir, img_name)
                try:
                    img = Image.open(img_path).convert('RGB')
                except:
                    img = Image.new('RGB', (224, 224), (128, 128, 128))
                batch_images.append(img)
            
            # Process images
            inputs = processor(images=batch_images, return_tensors="pt")
            pixel_values = inputs['pixel_values'].to(device)
            
            # Get predictions
            outputs = model(pixel_values)
            probs = torch.softmax(outputs, dim=1)
            all_probs.append(probs.cpu().numpy())
    
    return np.vstack(all_probs)

print("Prediction functions defined!")

## Generate Validation Predictions

In [ ]:
# =============================
# GENERATE VALIDATION PREDICTIONS
# =============================
print("Generating predictions on validation set...\n")

# Text predictions
val_texts = val_split['Processed_Text'].fillna('').tolist()
text_proba_val = get_text_predictions(val_texts, mbert_model, mbert_tokenizer, device)

# Image predictions
val_images = val_split['Image_name'].tolist()
image_proba_val = get_image_predictions(val_images, TRAIN_IMG_DIR, clip_model, clip_processor, device)

# True labels
y_val = (val_split['Label'] == 'Political').astype(int).values

print(f"\nValidation predictions generated!")
print(f"Text probabilities shape: {text_proba_val.shape}")
print(f"Image probabilities shape: {image_proba_val.shape}")
print(f"Labels shape: {y_val.shape}")

## Analyze Base Model Performance

In [ ]:
# =============================
# BASE MODEL PERFORMANCE
# =============================
text_preds_val = np.argmax(text_proba_val, axis=1)
image_preds_val = np.argmax(image_proba_val, axis=1)

text_acc = accuracy_score(y_val, text_preds_val)
text_f1 = f1_score(y_val, text_preds_val, average='macro')

image_acc = accuracy_score(y_val, image_preds_val)
image_f1 = f1_score(y_val, image_preds_val, average='macro')

print("=" * 60)
print("BASE MODEL PERFORMANCE ON VALIDATION SET")
print("=" * 60)
print(f"\nText Model (mBERT):")
print(f"  Accuracy: {text_acc:.4f}")
print(f"  F1 Score: {text_f1:.4f}")

print(f"\nImage Model (CLIP):")
print(f"  Accuracy: {image_acc:.4f}")
print(f"  F1 Score: {image_f1:.4f}")

print(f"\n{'='*60}")

#  Engineer Meta-Features for Stacking

In [ ]:
# =============================
# FEATURE ENGINEERING FOR META-MODEL
# =============================

def create_stacking_features(text_proba, image_proba):
    """
    Create engineered features for meta-model
    
    Features:
    1-2: Raw text probabilities [NonPol, Pol]
    3-4: Raw image probabilities [NonPol, Pol]
    5: Text confidence (max probability)
    6: Image confidence (max probability)
    7: Models agreement (binary: do they predict same class?)
    8: Disagreement magnitude (absolute difference in Political prob)
    9: Both predict Political (product of Political probs)
    10: Both predict NonPolitical (product of NonPolitical probs)
    11: Average Political probability
    12: Max Political probability
    13: Min Political probability
    14: Difference (Text Political - Image Political)
    """
    features = np.column_stack([
        # Base probabilities (4 features)
        text_proba[:, 0],              # Text: prob NonPolitical
        text_proba[:, 1],              # Text: prob Political
        image_proba[:, 0],             # Image: prob NonPolitical
        image_proba[:, 1],             # Image: prob Political
        
        # Confidence features (2 features)
        np.max(text_proba, axis=1),    # Text confidence
        np.max(image_proba, axis=1),   # Image confidence
        
        # Agreement features (2 features)
        (np.argmax(text_proba, axis=1) == np.argmax(image_proba, axis=1)).astype(float),  # Agreement
        np.abs(text_proba[:, 1] - image_proba[:, 1]),  # Disagreement magnitude
        
        # Interaction features (2 features)
        text_proba[:, 1] * image_proba[:, 1],  # Both say Political
        text_proba[:, 0] * image_proba[:, 0],  # Both say NonPolitical
        
        # Statistical features (4 features)
        (text_proba[:, 1] + image_proba[:, 1]) / 2,  # Average Political prob
        np.maximum(text_proba[:, 1], image_proba[:, 1]),  # Max Political prob
        np.minimum(text_proba[:, 1], image_proba[:, 1]),  # Min Political prob
        text_proba[:, 1] - image_proba[:, 1],  # Difference (Text - Image)
    ])
    
    return features

# Create validation features
X_val_stacking = create_stacking_features(text_proba_val, image_proba_val)

print(f"Stacking features shape: {X_val_stacking.shape}")
print(f"Total features: {X_val_stacking.shape[1]}")
print(f"\nFeature breakdown:")
print(f"  - Base probabilities: 4")
print(f"  - Confidence features: 2")
print(f"  - Agreement features: 2")
print(f"  - Interaction features: 2")
print(f"  - Statistical features: 4")
print(f"  - Total: 14 features")

# Train Meta-Model (Logistic Regression)

In [ ]:
# =============================
# TRAIN META-MODEL WITH GRID SEARCH
# =============================
print("Training Logistic Regression meta-model with Grid Search...\n")

# Define parameter grid
param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0, 100.0],  # Regularization strength
    'class_weight': ['balanced', None],
    'solver': ['lbfgs', 'liblinear']
}

# Create base meta-model
meta_model_base = LogisticRegression(max_iter=1000, random_state=42)

# Grid search with cross-validation
grid_search = GridSearchCV(
    meta_model_base,
    param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_val_stacking, y_val)

# Best model
meta_model = grid_search.best_estimator_

print(f"\n{'='*60}")
print("GRID SEARCH RESULTS")
print(f"{'='*60}")
print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best CV F1 Score: {grid_search.best_score_:.4f}")
print(f"\n{'='*60}")

## Analyze Meta-Model Performance

In [ ]:
# =============================
# META-MODEL PERFORMANCE
# =============================
ensemble_preds_val = meta_model.predict(X_val_stacking)
ensemble_proba_val = meta_model.predict_proba(X_val_stacking)

ensemble_acc = accuracy_score(y_val, ensemble_preds_val)
ensemble_f1 = f1_score(y_val, ensemble_preds_val, average='macro')

print("=" * 60)
print("ENSEMBLE MODEL PERFORMANCE")
print("=" * 60)
print(f"\nEnsemble (Stacking):")
print(f"  Accuracy: {ensemble_acc:.4f}")
print(f"  F1 Score: {ensemble_f1:.4f}")

print(f"\n{'='*60}")
print("PERFORMANCE COMPARISON")
print(f"{'='*60}")
print(f"\nText Model:     Acc={text_acc:.4f}, F1={text_f1:.4f}")
print(f"Image Model:    Acc={image_acc:.4f}, F1={image_f1:.4f}")
print(f"Ensemble:       Acc={ensemble_acc:.4f}, F1={ensemble_f1:.4f}")

print(f"\n{'='*60}")
print("IMPROVEMENTS")
print(f"{'='*60}")
print(f"Accuracy improvement over Text:  {(ensemble_acc - text_acc)*100:+.2f}%")
print(f"Accuracy improvement over Image: {(ensemble_acc - image_acc)*100:+.2f}%")
print(f"F1 improvement over Text:        {(ensemble_f1 - text_f1)*100:+.2f}%")
print(f"F1 improvement over Image:       {(ensemble_f1 - image_f1)*100:+.2f}%")
print(f"\n{'='*60}")

# Late Fusion
- Between first model build with deep learing and stacking model build with logistic regresion


# =============================
# LATE FUSION: Combining Neural Network & Ensemble Predictions
# =============================
print("="*60)
print("LATE FUSION IMPLEMENTATION")
print("="*60)
print("\nCombining predictions from:")
print("  1. Deep Learning Model (Neural Network on text features)")
print("  2. Stacking Ensemble (mBERT + CLIP with Logistic Regression)")
print()

### STEP 1: Load Neural Network Model & Get Predictions

In [ ]:
print("Step 1: Loading Neural Network model...\n")

# Load the trained neural network
from tensorflow import keras
nn_model = keras.models.load_model('neural_network_model.h5')
print("✓ Neural Network model loaded!")

# Get validation predictions from Neural Network
# Use the same validation set as the ensemble
val_features_nn = val_split[['political_specific_count', 'political_specific_ratio']].fillna(0)
val_features_nn_scaled = scaler.transform(val_features_nn)

nn_proba_val = nn_model.predict(val_features_nn_scaled, verbose=0)
nn_preds_val = (nn_proba_val > 0.5).astype(int).flatten()

# Get test predictions from Neural Network
test_features_nn = test_text_df[['political_specific_count', 'political_specific_ratio']].fillna(0)
test_features_nn_scaled = scaler.transform(test_features_nn)

nn_proba_test = nn_model.predict(test_features_nn_scaled, verbose=0)
nn_preds_test = (nn_proba_test > 0.5).astype(int).flatten()

print(f"\nNeural Network predictions generated!")
print(f"  Val shape: {nn_proba_val.shape}")
print(f"  Test shape: {nn_proba_test.shape}")

### STEP 2: Get Ensemble Predictions

In [ ]:
print("\nStep 2: Using existing Ensemble predictions...")
print(f"  Ensemble val predictions: {ensemble_proba_val.shape}")
print(f"  Ensemble test predictions: {ensemble_proba_test.shape}")

### STEP 3: Analyze Individual Model Performance on Validation

In [ ]:
print("\n" + "="*60)
print("INDIVIDUAL MODEL PERFORMANCE (Validation Set)")
print("="*60)

nn_acc_val = accuracy_score(y_val, nn_preds_val)
nn_f1_val = f1_score(y_val, nn_preds_val, average='macro')

print(f"\n1. Neural Network (Text Features):")
print(f"     Accuracy: {nn_acc_val:.4f}")
print(f"     F1 Score: {nn_f1_val:.4f}")

print(f"\n2. Text Model (mBERT):")
print(f"     Accuracy: {text_acc:.4f}")
print(f"     F1 Score: {text_f1:.4f}")

print(f"\n3. Image Model (CLIP):")
print(f"     Accuracy: {image_acc:.4f}")
print(f"     F1 Score: {image_f1:.4f}")

print(f"\n4. Ensemble (Stacking):")
print(f"     Accuracy: {ensemble_acc:.4f}")
print(f"     F1 Score: {ensemble_f1:.4f}")

### STEP 4: Late Fusion Methods

In [ ]:
print("\n" + "="*60)
print("LATE FUSION STRATEGIES")
print("="*60)

# Convert probabilities to same format (both should be [N, 2] shape)
# Neural Network outputs [N, 1], so we need to convert
nn_proba_val_2d = np.column_stack([1 - nn_proba_val, nn_proba_val])
nn_proba_test_2d = np.column_stack([1 - nn_proba_test, nn_proba_test])

# Method 1: Simple Average (Equal Weights)
print("\n1. SIMPLE AVERAGE (Equal Weights)")
avg_proba_val = (nn_proba_val_2d + ensemble_proba_val) / 2
avg_preds_val = np.argmax(avg_proba_val, axis=1)

avg_acc = accuracy_score(y_val, avg_preds_val)
avg_f1 = f1_score(y_val, avg_preds_val, average='macro')
print(f"   Validation Accuracy: {avg_acc:.4f}")
print(f"   Validation F1 Score: {avg_f1:.4f}")

# Method 2: Weighted Average (Based on Validation Performance)
print("\n2. WEIGHTED AVERAGE (Performance-based)")
# Weights based on F1 scores
total_f1 = nn_f1_val + ensemble_f1
w_nn = nn_f1_val / total_f1
w_ensemble = ensemble_f1 / total_f1

weighted_proba_val = w_nn * nn_proba_val_2d + w_ensemble * ensemble_proba_val
weighted_preds_val = np.argmax(weighted_proba_val, axis=1)

weighted_acc = accuracy_score(y_val, weighted_preds_val)
weighted_f1 = f1_score(y_val, weighted_preds_val, average='macro')
print(f"   Weights: NN={w_nn:.3f}, Ensemble={w_ensemble:.3f}")
print(f"   Validation Accuracy: {weighted_acc:.4f}")
print(f"   Validation F1 Score: {weighted_f1:.4f}")

# Method 3: Max Probability Voting
print("\n3. MAX PROBABILITY VOTING")
max_nn = np.max(nn_proba_val_2d, axis=1)
max_ensemble = np.max(ensemble_proba_val, axis=1)

max_vote_preds_val = np.where(
    max_nn > max_ensemble,
    np.argmax(nn_proba_val_2d, axis=1),
    np.argmax(ensemble_proba_val, axis=1)
)

max_vote_acc = accuracy_score(y_val, max_vote_preds_val)
max_vote_f1 = f1_score(y_val, max_vote_preds_val, average='macro')
print(f"   Validation Accuracy: {max_vote_acc:.4f}")
print(f"   Validation F1 Score: {max_vote_f1:.4f}")

# Method 4: Majority Voting
print("\n4. MAJORITY VOTING")
nn_vote = nn_preds_val
ensemble_vote = ensemble_preds_val

majority_preds_val = np.where(
    nn_vote == ensemble_vote,
    nn_vote,  # If they agree, use their prediction
    ensemble_vote  # If they disagree, trust ensemble
)

majority_acc = accuracy_score(y_val, majority_preds_val)
majority_f1 = f1_score(y_val, majority_preds_val, average='macro')
print(f"   Validation Accuracy: {majority_acc:.4f}")
print(f"   Validation F1 Score: {majority_f1:.4f}")

# Method 5: Learned Fusion (Train another meta-model)
print("\n5. LEARNED FUSION (Meta-Logistic Regression)")
fusion_features_val = np.column_stack([
    nn_proba_val_2d[:, 1],  # NN Political probability
    ensemble_proba_val[:, 1],  # Ensemble Political probability
    nn_proba_val_2d[:, 1] * ensemble_proba_val[:, 1],  # Product
    np.abs(nn_proba_val_2d[:, 1] - ensemble_proba_val[:, 1]),  # Difference
    np.max(nn_proba_val_2d, axis=1),  # NN confidence
    np.max(ensemble_proba_val, axis=1),  # Ensemble confidence
])

fusion_model = LogisticRegression(max_iter=1000, random_state=42)
fusion_model.fit(fusion_features_val, y_val)
fusion_preds_val = fusion_model.predict(fusion_features_val)

fusion_acc = accuracy_score(y_val, fusion_preds_val)
fusion_f1 = f1_score(y_val, fusion_preds_val, average='macro')
print(f"   Validation Accuracy: {fusion_acc:.4f}")
print(f"   Validation F1 Score: {fusion_f1:.4f}")

### STEP 5: Compare All Methods

In [ ]:
print("\n" + "="*60)
print("PERFORMANCE COMPARISON")
print("="*60)

results = pd.DataFrame({
    'Model': [
        'Neural Network',
        'Ensemble (Stacking)',
        'Late Fusion: Average',
        'Late Fusion: Weighted',
        'Late Fusion: Max Prob',
        'Late Fusion: Majority',
        'Late Fusion: Learned'
    ],
    'Accuracy': [nn_acc_val, ensemble_acc, avg_acc, weighted_acc, max_vote_acc, majority_acc, fusion_acc],
    'F1 Score': [nn_f1_val, ensemble_f1, avg_f1, weighted_f1, max_vote_f1, majority_f1, fusion_f1]
})

print(results.to_string(index=False))

# Find best method
best_method_idx = results['F1 Score'].idxmax()
best_method_name = results.iloc[best_method_idx]['Model']
best_f1 = results.iloc[best_method_idx]['F1 Score']

print(f"\n🏆 BEST METHOD: {best_method_name} (F1: {best_f1:.4f})")

### STEP 6: Visualize Results

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart for comparison
results_sorted = results.sort_values('F1 Score', ascending=True)
colors = ['#3498db' if 'Late Fusion' not in m else '#e74c3c' for m in results_sorted['Model']]

ax1.barh(results_sorted['Model'], results_sorted['F1 Score'], color=colors)
ax1.set_xlabel('F1 Score')
ax1.set_title('Model Performance Comparison')
ax1.axvline(x=ensemble_f1, color='green', linestyle='--', linewidth=2, label='Ensemble Baseline')
ax1.legend()
ax1.grid(axis='x', alpha=0.3)

# Confusion matrix for best method
if best_method_name == 'Late Fusion: Average':
    best_preds = avg_preds_val
elif best_method_name == 'Late Fusion: Weighted':
    best_preds = weighted_preds_val
elif best_method_name == 'Late Fusion: Max Prob':
    best_preds = max_vote_preds_val
elif best_method_name == 'Late Fusion: Majority':
    best_preds = majority_preds_val
else:
    best_preds = fusion_preds_val

cm = confusion_matrix(y_val, best_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='RdYlGn', ax=ax2,
            xticklabels=['NonPol', 'Pol'], yticklabels=['NonPol', 'Pol'])
ax2.set_title(f'Confusion Matrix - {best_method_name}')
ax2.set_ylabel('True Label')
ax2.set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig('late_fusion_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Comparison plot saved as 'late_fusion_comparison.png'")

### STEP 7: Generate Test Predictions with Best Method

In [ ]:
print("\n" + "="*60)
print("GENERATING TEST PREDICTIONS WITH BEST METHOD")
print("="*60)

# Apply best method to test set
if best_method_name == 'Late Fusion: Average':
    test_proba_late_fusion = (nn_proba_test_2d + ensemble_proba_test) / 2
    
elif best_method_name == 'Late Fusion: Weighted':
    test_proba_late_fusion = w_nn * nn_proba_test_2d + w_ensemble * ensemble_proba_test
    
elif best_method_name == 'Late Fusion: Max Prob':
    max_nn_test = np.max(nn_proba_test_2d, axis=1)
    max_ensemble_test = np.max(ensemble_proba_test, axis=1)
    test_preds_late_fusion = np.where(
        max_nn_test > max_ensemble_test,
        np.argmax(nn_proba_test_2d, axis=1),
        np.argmax(ensemble_proba_test, axis=1)
    )
    
elif best_method_name == 'Late Fusion: Majority':
    nn_vote_test = nn_preds_test
    ensemble_vote_test = np.argmax(ensemble_proba_test, axis=1)
    test_preds_late_fusion = np.where(
        nn_vote_test == ensemble_vote_test,
        nn_vote_test,
        ensemble_vote_test
    )
    
else:  # Learned Fusion
    fusion_features_test = np.column_stack([
        nn_proba_test_2d[:, 1],
        ensemble_proba_test[:, 1],
        nn_proba_test_2d[:, 1] * ensemble_proba_test[:, 1],
        np.abs(nn_proba_test_2d[:, 1] - ensemble_proba_test[:, 1]),
        np.max(nn_proba_test_2d, axis=1),
        np.max(ensemble_proba_test, axis=1),
    ])
    test_preds_late_fusion = fusion_model.predict(fusion_features_test)

# If we have probabilities, convert to predictions
if 'test_proba_late_fusion' in locals():
    test_preds_late_fusion = np.argmax(test_proba_late_fusion, axis=1)

print(f"\n✓ Using: {best_method_name}")
print(f"  Test predictions generated: {len(test_preds_late_fusion)}")
print(f"  Political: {(test_preds_late_fusion == 1).sum()}")
print(f"  NonPolitical: {(test_preds_late_fusion == 0).sum()}")

### STEP 8: Create Final Submission

In [ ]:
submission_late_fusion = pd.DataFrame({
    'Image_name': test_text_df['Image_name'],
    'Label': ['Political' if pred == 1 else 'NonPolitical' for pred in test_preds_late_fusion]
})

submission_late_fusion.to_csv('submission_late_fusion.csv', index=False)

print("\n" + "="*60)
print("FINAL SUBMISSION CREATED")
print("="*60)
print(f"\nFile: submission_late_fusion.csv")
print(f"Method: {best_method_name}")
print(f"Expected Performance (from validation):")
print(f"  Accuracy: {best_f1:.4f}")
print(f"  F1 Score: {best_f1:.4f}")
print(f"\nPrediction distribution:")
print(submission_late_fusion['Label'].value_counts())
print(f"\nFirst 10 predictions:")
print(submission_late_fusion.head(10))
print(f"\n{'='*60}")
print(f"{'='*60}")

### STEP 9: Summary Report

In [ ]:
print("\n" + "="*60)
print("LATE FUSION - FINAL SUMMARY")
print("="*60)

print("\n MODEL ARCHITECTURE:")
print("   Model 1: Neural Network (3 hidden layers)")
print("           - Input: political_specific_count, political_specific_ratio")
print("           - Architecture: 64→32→16→1 neurons")
print("   ")
print("   Model 2: Stacking Ensemble")
print("           - Base models: mBERT (text) + CLIP (image)")
print("           - Meta-model: Logistic Regression")
print("           - Features: 14 engineered features")

print("\n PERFORMANCE PROGRESSION:")
print(f"   Neural Network:     F1={nn_f1_val:.4f}")
print(f"   Ensemble:           F1={ensemble_f1:.4f}")
print(f"   Late Fusion (Best): F1={best_f1:.4f}")
print(f"   ")
print(f"   Improvement: +{(best_f1 - max(nn_f1_val, ensemble_f1))*100:.2f}%")

print("\n LATE FUSION METHOD:")
print(f"   Selected: {best_method_name}")
print(f"   Combines: Neural Network + Stacking Ensemble")

print("\n OUTPUT FILES:")
print("   1. submission_late_fusion.csv (FINAL - BEST)")
print("   2. late_fusion_comparison.png")

print(f"\n{'='*60}")